In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Customer churn prediction').getOrCreate()
spark

In [3]:
df = spark.read.csv('C:/Users/md.owais/Desktop/Customer Chun Prediction/data/test_churn.csv', inferSchema=True, header=True)

In [4]:
df.show()

+---+-----+--------------+-------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+
| id|state|account_length|    area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|
+---+-----+--------------+-------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------

Shape

In [5]:
shape = df.count(), len(df.columns)
shape

(750, 20)

Mising Values Analysis

In [6]:
from pyspark.sql.functions import isnan, count, when, col, expr, substring
total_col = df.columns
df.select([count(when( col(c).contains('NA') | col(c).contains('None') \
                      | (col(c) == "NaN") | (col(c) == '') | (col(c) == 'NULL')\
                      | col(c).isNull() | isnan(c) ,c)).alias(c) for c in total_col]).show()

+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+-----------------------------+
| id|state|account_length|area_code|international_plan|voice_mail_plan|number_vmail_messages|total_day_minutes|total_day_calls|total_day_charge|total_eve_minutes|total_eve_calls|total_eve_charge|total_night_minutes|total_night_calls|total_night_charge|total_intl_minutes|total_intl_calls|total_intl_charge|number_customer_service_calls|
+---+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------------------

Duplicates Records analysis

In [7]:
df.count(), df.distinct().count()

(750, 750)

EDA

In [8]:
# 1. Remove area_code_ from area_code column
# Remove id column


In [9]:
df=df.drop('id')